# 使用Redis作为OpenAI Chat的上下文存储本笔记演示了如何使用Redis作为ChatGPT的高速上下文存储。## 先决条件* 带有Redis Search和Redis JSON模块的Redis实例* Redis-py客户端库* OpenAI Python客户端库* OpenAI API密钥## 安装安装示例所需的Python模块。

In [ ]:
! pip install redis openai python-dotenv openai[datalib]

## OpenAI API密钥创建一个.env文件并将您的OpenAI密钥添加到其中

In [ ]:
OPENAI_API_KEY=your_key

## OpenAI设置加载密钥 + 用于聊天完成的辅助函数

In [ ]:
import openaiimport osfrom dotenv import load_dotenvload_dotenv()openai.api_key = os.getenv("OPENAI_API_KEY")def get_completion(prompt, model="gpt-3.5-turbo"):    messages = [{"role": "user", "content": prompt}]    response = openai.ChatCompletion.create(        model=model,        messages=messages,        temperature=0,     )    return response.choices[0].message["content"]

## 实验 - 在模型知识截止日期之外的主题上进行聊天补全Gpt-3.5-turbo是在2021年9月之前的数据上进行训练的。让我们问一个关于那个日期之后的事情的问题。在这种情况下，是关于FTX/Sam Bankman-Fried丑闻的问题。

In [ ]:
prompt = "Is Sam Bankman-Fried's company, FTX, considered a well-managed company?"response = get_completion(prompt)print(response)

## 信息不完整这些AI系统的一个不幸行为是，即使系统对其结果不够自信，系统也会提供听起来很自信的响应。缓解这种情况的一种方法是进行提示重新设计，如下所示。

In [ ]:
prompt ="Is Sam Bankman-Fried's company, FTX, considered a well-managed company?  If you don't know for certain, say unknown."response = get_completion(prompt)print(response)

## 附加背景另一种解决不完整信息的方法是提供系统更多信息，使其能够做出智能决策而不是猜测。我们将使用Redis作为提供额外背景信息的来源。我们将引入GPT知识截止日期之后的商业新闻文章，以便系统能更好地了解FTX是如何实际管理的。

## 启动Redis Stack Docker容器

In [ ]:
! docker compose up -d

## 连接Redis客户端

In [11]:
from redis import from_urlREDIS_URL = 'redis://localhost:6379'client = from_url(REDIS_URL)client.ping()

True

## 创建索引[FT.CREATE](https://redis.io/commands/ft.create/)

In [12]:
from redis.commands.search.field import TextField, VectorFieldfrom redis.commands.search.indexDefinition import IndexDefinition, IndexTypeschema = [ VectorField('$.vector',             "FLAT",             {   "TYPE": 'FLOAT32',                 "DIM": 1536,                 "DISTANCE_METRIC": "COSINE"            },  as_name='vector' ),            TextField('$.content', as_name='content')        ]idx_def = IndexDefinition(index_type=IndexType.JSON, prefix=['doc:'])try:     client.ft('idx').dropindex()except:    passclient.ft('idx').create_index(schema, definition=idx_def)

b'OK'

## 将数据文件加载到Redis中，作为具有文本和向量字段的JSON对象[Redis JSON](https://redis.io/docs/stack/json/)

In [13]:
import osimport openaidirectory = './assets/'model='text-embedding-3-small'i = 1for file in os.listdir(directory):    with open(os.path.join(directory, file)) as f:        content = f.read()        vector = openai.Embedding.create(input = [content], model = model)['data'][0]['embedding']        client.json().set(f'doc:{i}', '$', {'content': content, 'vector': vector})    i += 1

## 嵌入问题并执行VSS以找到最相关的文档[KNN搜索](https://redis.io/docs/stack/search/reference/vectors/#knn-search)

In [14]:
from redis.commands.search.query import Queryimport numpy as npvec = np.array(openai.Embedding.create(input = [prompt], model = model)['data'][0]['embedding'], dtype=np.float32).tobytes()q = Query('*=>[KNN 1 @vector $query_vec AS vector_score]')\    .sort_by('vector_score')\    .return_fields('content')\    .dialect(2)    params = {"query_vec": vec}context = client.ft('idx').search(q, query_params=params).docs[0].contentprint(context)

Embattled Crypto Exchange FTX Files for Bankruptcy

Nov. 11, 2022
On Monday, Sam Bankman-Fried, the chief executive of the cryptocurrency exchange FTX, took to Twitter to reassure his customers: “FTX is fine,” he wrote. “Assets are fine.”

On Friday, FTX announced that it was filing for bankruptcy, capping an extraordinary week of corporate drama that has upended crypto markets, sent shock waves through an industry struggling to gain mainstream credibility and sparked government investigations that could lead to more damaging revelations or even criminal charges.

In a statement on Twitter, the company said that Mr. Bankman-Fried had resigned, with John J. Ray III, a corporate turnaround specialist, taking over as chief executive.

The speed of FTX’s downfall has left crypto insiders stunned. Just days ago, Mr. Bankman-Fried was considered one of the smartest leaders in the crypto industry, an influential figure in Washington who was lobbying to shape regulations. And FTX was widely vi

## 重复带有上下文的问题给OpenAI现在我们有了相关的上下文，请将其添加到提示中发送给OpenAI，并获得一个非常不同的回复。

In [15]:
prompt = f"""根据提供的信息，Sam Bankman-Fried的公司FTX被认为是一家管理良好的公司。"""response = get_completion(prompt)print(response)

No, Sam Bankman-Fried's company FTX is not considered a well-managed company as it has filed for bankruptcy and owes as much as $8 billion to its creditors. The collapse of FTX has destabilized the crypto industry, and the company is already the target of investigations by the Securities and Exchange Commission and the Justice Department. FTX was widely viewed as one of the most stable and responsible companies in the freewheeling, loosely regulated crypto industry, but its risky practices have become pervasive in crypto, leading to a reckoning.
